In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv('charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME","ORGANIZATION","ASK_AMT"],1)

C:\Users\asilo\AppData\Local\Temp/ipykernel_10008/1134476541.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(["EIN","NAME","ORGANIZATION","ASK_AMT"],1)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
STATUS                     2
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
application_types_to_replace = list(counts[counts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
counts_class = application_df['CLASSIFICATION'].value_counts()
counts_class

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
classication_types_to_replace = list(counts_class[counts_class<500].index)
classication_types_to_replace

# Replace in dataframe
for cls in classication_types_to_replace:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df= application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim = number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3200      
                                                                 
 dense_1 (Dense)             (None, 20)                1620      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4,841
Trainable params: 4,841
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 737us/step - loss: 0.6117 - accuracy: 0.6999
Epoch 2/100
804/804 [==============================] - 1s 738us/step - loss: 0.5868 - accuracy: 0.7238
Epoch 3/100
804/804 [==============================] - 1s 770us/step - loss: 0.5772 - accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 719us/step - loss: 0.5893 - accuracy: 0.7127
Epoch 5/100
804/804 [==============================] - 1s 724us/step - loss: 0.5760 - accuracy: 0.7237
Epoch 6/100
804/804 [==============================] - 1s 725us/step - loss: 0.5748 - accuracy: 0.7264
Epoch 7/100
804/804 [==============================] - 1s 724us/step - loss: 0.5739 - accuracy: 0.7279
Epoch 8/100
804/804 [==============================] - 1s 723us/step - loss: 0.5749 - accuracy: 0.7273
Epoch 9/100
804/804 [==============================] - 1s 729us/step - loss: 0.5956 - accuracy: 0.7136
Epoch 10/100
804/804 [==============================] - 1s 743us/step - l

804/804 [==============================] - 1s 744us/step - loss: 0.5544 - accuracy: 0.7317
Epoch 80/100
804/804 [==============================] - 1s 758us/step - loss: 0.5534 - accuracy: 0.7311
Epoch 81/100
804/804 [==============================] - 1s 715us/step - loss: 0.5533 - accuracy: 0.7321
Epoch 82/100
804/804 [==============================] - 1s 720us/step - loss: 0.5541 - accuracy: 0.7323
Epoch 83/100
804/804 [==============================] - 1s 715us/step - loss: 0.5542 - accuracy: 0.7316
Epoch 84/100
804/804 [==============================] - 1s 722us/step - loss: 0.5561 - accuracy: 0.7318
Epoch 85/100
804/804 [==============================] - 1s 713us/step - loss: 0.5534 - accuracy: 0.7314
Epoch 86/100
804/804 [==============================] - 1s 724us/step - loss: 0.5531 - accuracy: 0.7318
Epoch 87/100
804/804 [==============================] - 1s 722us/step - loss: 0.5548 - accuracy: 0.7323
Epoch 88/100
804/804 [==============================] - 1s 737us/step - loss:

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5624 - accuracy: 0.7233 - 232ms/epoch - 866us/step
Loss: 0.5623865723609924, Accuracy: 0.7232652902603149


In [18]:
#OPTIMIZATION 2

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim = number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))

# 3rd hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3200      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7,281
Trainable params: 7,281
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 729us/step - loss: 0.9913 - accuracy: 0.7284
Epoch 2/100
804/804 [==============================] - 1s 714us/step - loss: 1.0029 - accuracy: 0.7281
Epoch 3/100
804/804 [==============================] - 1s 714us/step - loss: 1.0301 - accuracy: 0.7286
Epoch 4/100
804/804 [==============================] - 1s 738us/step - loss: 1.1171 - accuracy: 0.7278
Epoch 5/100
804/804 [==============================] - 1s 709us/step - loss: 1.1029 - accuracy: 0.7285
Epoch 6/100
804/804 [==============================] - 1s 741us/step - loss: 1.0783 - accuracy: 0.7298
Epoch 7/100
804/804 [==============================] - 1s 705us/step - loss: 1.0890 - accuracy: 0.7285
Epoch 8/100
804/804 [==============================] - 1s 724us/step - loss: 1.0717 - accuracy: 0.7289
Epoch 9/100
804/804 [==============================] - 1s 707us/step - loss: 1.1549 - accuracy: 0.7297
Epoch 10/100
804/804 [==============================] - 1s 709us/step - l

804/804 [==============================] - 1s 714us/step - loss: 0.9955 - accuracy: 0.7311
Epoch 80/100
804/804 [==============================] - 1s 716us/step - loss: 1.1873 - accuracy: 0.7313
Epoch 81/100
804/804 [==============================] - 1s 702us/step - loss: 1.1764 - accuracy: 0.7307
Epoch 82/100
804/804 [==============================] - 1s 722us/step - loss: 1.2546 - accuracy: 0.7307
Epoch 83/100
804/804 [==============================] - 1s 702us/step - loss: 1.3253 - accuracy: 0.7315
Epoch 84/100
804/804 [==============================] - 1s 703us/step - loss: 1.3054 - accuracy: 0.7316
Epoch 85/100
804/804 [==============================] - 1s 707us/step - loss: 1.2425 - accuracy: 0.7296
Epoch 86/100
804/804 [==============================] - 1s 714us/step - loss: 1.2331 - accuracy: 0.7317
Epoch 87/100
804/804 [==============================] - 1s 707us/step - loss: 1.2865 - accuracy: 0.7305
Epoch 88/100
804/804 [==============================] - 1s 699us/step - loss:

In [23]:
#OPTIMIZATION 3

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="sigmoid", input_dim = number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 40)                1600      
                                                                 
 dense_8 (Dense)             (None, 20)                820       
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 692us/step - loss: 0.6805 - accuracy: 0.7021
Epoch 2/100
804/804 [==============================] - 1s 669us/step - loss: 0.6431 - accuracy: 0.7206
Epoch 3/100
804/804 [==============================] - 1s 690us/step - loss: 0.6347 - accuracy: 0.7245
Epoch 4/100
804/804 [==============================] - 1s 676us/step - loss: 0.6181 - accuracy: 0.7231
Epoch 5/100
804/804 [==============================] - 1s 669us/step - loss: 0.6030 - accuracy: 0.7248
Epoch 6/100
804/804 [==============================] - 1s 702us/step - loss: 0.6050 - accuracy: 0.7260
Epoch 7/100
804/804 [==============================] - 1s 677us/step - loss: 0.6052 - accuracy: 0.7262
Epoch 8/100
804/804 [==============================] - 1s 706us/step - loss: 0.6038 - accuracy: 0.7252
Epoch 9/100
804/804 [==============================] - 1s 665us/step - loss: 0.5857 - accuracy: 0.7258
Epoch 10/100
804/804 [==============================] - 1s 663us/step - l

804/804 [==============================] - 1s 667us/step - loss: 0.6548 - accuracy: 0.7297
Epoch 80/100
804/804 [==============================] - 1s 665us/step - loss: 0.6569 - accuracy: 0.7296
Epoch 81/100
804/804 [==============================] - 1s 663us/step - loss: 0.6473 - accuracy: 0.7304
Epoch 82/100
804/804 [==============================] - 1s 667us/step - loss: 0.6369 - accuracy: 0.7302
Epoch 83/100
804/804 [==============================] - 1s 661us/step - loss: 0.6381 - accuracy: 0.7301
Epoch 84/100
804/804 [==============================] - 1s 667us/step - loss: 0.6347 - accuracy: 0.7313
Epoch 85/100
804/804 [==============================] - 1s 695us/step - loss: 0.6943 - accuracy: 0.7300
Epoch 86/100
804/804 [==============================] - 1s 669us/step - loss: 0.6670 - accuracy: 0.7290
Epoch 87/100
804/804 [==============================] - 1s 668us/step - loss: 0.6600 - accuracy: 0.7306
Epoch 88/100
804/804 [==============================] - 1s 665us/step - loss:

In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")